We have manually selected some ROI centers by inspecting the classification maps. Then, we set a sphere (radius in voxel = 3) around those centers and select the voxel containing maximum accuracy. That voxel will be then the center of a sphere of radius 5 voxels that will be used as ROI.

In [1]:
from mvpa2.base.hdf5 import h5load
from mvpa2.datasets.mri import fmri_dataset, Dataset
from mvpa2.suite import Sphere, IndexQueryEngine
import numpy as np
from os.path import join as pjoin
from mvpa2.datasets.mri import map2nifti

In [2]:
basedir = '/data/famface/openfmri/results/l2ants_final/model001/task001/subjects_all/'
anatype_fam = 'slmskz5vx_svm/familiar_vs_unfamiliar-id'
anatype_id = 'slmskz5vx_svm/identity-familiar+identity-unfamiliar'

In [3]:
ds_fam = fmri_dataset(pjoin(basedir, 'stats/mvpa', anatype_fam, 'avg_sl_map.nii.gz'))
ds_id = fmri_dataset(pjoin(basedir, 'stats/mvpa', anatype_id, 'avg_sl_map.nii.gz'))
mask = fmri_dataset('/data/famface/openfmri/results/l2ants_final/model001/task001/subjects_all/mask/union_mask_33sbjs_80p.nii.gz')
mask_ = mask.samples[0] > 0
ds_fam = ds_fam[:, mask_]
ds_id = ds_id[:, mask_]

 * Please note: warnings are printed only once, but underlying problem might occur many times *


In [30]:
# rois within stat maps of familiar and identity classification
rois = {
'lvPreCun': [-8, -56, 26],
'ldPreCun': [-8, -70, 34],
'ldPreCun-Id': [-10, -62, 52],
'lLingual': [-8, -82, -8],
'lEV-Id': [-10, -94, -4],
'lpFFA': [-46, -42, -20],
'laFFA': [-40, -14, -28],
'lIFG': [-42, 46, 4],
'lIFG-Id': [-46, 34, -6],
'lTPJ': [-56, -60, 26],
'lpMTG': [-60, -48, 4],
'lmMTG': [-60, -28, -8],
'laMTG': [-54, 2, -22],
'lParaCin': [-8, 48, -6],
'rvPreCun': [10, -58, 18],
'rdPreCun': [8, -70, 34],
'rdPreCun-Id': [10, -62, 52],
'rLingual': [10, -74, -4],
'rEV-Id': [10, -90, 2],
'rpFFA': [42, -40, -20],
'raFFA': [40, -14, -28],
'rTPJ': [56, -56, 24],
'rpMTG': [60, -48, 4],
'rmMTG': [60, -28, -2],
'raMTG': [60, -10, -8],
'rIFG': [56, 30, 2],
'rATL-Id': [36, 6, -36],
'raParaCin': [12, 46, 16],
'rFFA-Id': [40, -60, -16],
'lFFA-Id': [-40, -60, -16],
'rOFA-Id': [26, -84, -12],
'lOFA-Id': [-28, -84, -12],
}

In [31]:
print("Using {0} ROIs".format(len(rois)))

Using 32 ROIs


In [32]:
# this radius is just to find the maximum
radius_max = 3
sphere = Sphere(radius_max)

In [33]:
qe = IndexQueryEngine(voxel_indices=sphere)

In [34]:
qe.train(ds_fam)

In [35]:
invaffine = np.linalg.inv(ds_fam.a.imgaffine)
def xyz2ijk(xyz):
    xyz1 = np.hstack((xyz, [1]))
    ijk = np.dot(invaffine, xyz1)[:3].astype(int)
    return ijk

In [36]:
rois_ijk = {key: xyz2ijk(xyz) for key, xyz in rois.iteritems()}

In [37]:
xyz2ijk((0, 0, 0)), xyz2ijk((2, 0, 0)), xyz2ijk((-2, 0, 0))

(array([45, 63, 36]), array([44, 63, 36]), array([46, 63, 36]))

In [38]:
def filter_voxel_indices(nbr, vxl_indices, center_coord):
    # 45 is the i coordinate for x = 0mm
    sign_center = np.sign(center_coord[0] - 45)
    sign_vxl_indices = np.sign(vxl_indices[:, 0] - 45)
    return np.asarray(nbr)[sign_center * sign_vxl_indices > 0]

In [39]:
#taking the accuracy
rois_max = dict()
for roi, roi_coord in rois_ijk.iteritems():
    # select the correct dataset
    if roi.endswith('-Id'):
        which_ds = ds_id
    else:
        which_ds = ds_fam
    idx_center = np.where((which_ds.fa.voxel_indices == roi_coord).all(axis=1))[0][0]
    nbr = qe.query(voxel_indices=roi_coord)
    # now we need to filter taking ONLY the voxels in the same hemisphere
    voxel_indices_nbr = which_ds.fa.voxel_indices[nbr]
    nbr = filter_voxel_indices(nbr, voxel_indices_nbr, roi_coord)
    nbr_max = np.argmax(which_ds[:, nbr].samples[0])
    idx_max = nbr[nbr_max]
    if which_ds[:, idx_max].samples == which_ds[:, idx_center].samples:
        idx_max = idx_center
        print "Using idx_center for {0}".format(roi)
    rois_max[roi] = which_ds.fa.voxel_indices[idx_max]

Using idx_center for lEV-Id


In [40]:
# now make a bigger qe
qe_radius = 5
sphere = Sphere(qe_radius)
qe = IndexQueryEngine(voxel_indices=sphere)
qe.train(ds_fam)

In [41]:
rois_ = dict()
for roi, roi_coord in rois_max.iteritems():
    print("Querying {0}: {1}".format(roi, roi_coord))
    nbr = qe.query(voxel_indices=roi_coord)
    # filter so we keep only voxels in the same hemisphere
    nbr = filter_voxel_indices(nbr, ds_fam.fa.voxel_indices[nbr], 
                               roi_coord)
    rois_[roi] = (roi_coord, nbr)

from mvpa2.base.hdf5 import h5save
h5save('manual_rois_20170102.hdf5', rois_)

Querying rLingual: [42 25 34]
Querying lParaCin: [48 87 32]
Querying rIFG: [15 77 39]
Querying raParaCin: [39 87 46]
Querying lTPJ: [74 32 48]
Querying rEV-Id: [40 20 35]
Querying lpFFA: [69 41 26]
Querying rOFA-Id: [34 23 30]
Querying ldPreCun: [48 30 51]
Querying lFFA-Id: [66 34 28]
Querying laFFA: [66 58 24]
Querying lpMTG: [78 39 38]
Querying lEV-Id: [50 16 34]
Querying lmMTG: [77 49 33]
Querying raMTG: [14 56 30]
Querying rATL-Id: [27 66 20]
Querying raFFA: [25 57 21]
Querying rpFFA: [26 44 24]
Querying rTPJ: [19 37 47]
Querying lvPreCun: [47 35 48]
Querying rpMTG: [13 39 36]
Querying rdPreCun-Id: [42 31 61]
Querying lIFG: [66 85 40]
Querying rmMTG: [15 46 35]
Querying rvPreCun: [41 36 47]
Querying rFFA-Id: [23 31 29]
Querying lOFA-Id: [61 23 29]
Querying lLingual: [47 20 33]
Querying laMTG: [73 62 26]
Querying ldPreCun-Id: [48 30 61]
Querying lIFG-Id: [68 82 34]
Querying rdPreCun: [43 30 52]


In [42]:
for roi, (center, ids) in rois_.iteritems():
    print("{0} ({2}): {1}".format(roi, len(ids), center))

rLingual ([42 25 34]): 436
lParaCin ([48 87 32]): 436
rIFG ([15 77 39]): 265
raParaCin ([39 87 46]): 515
lLingual ([47 20 33]): 367
rEV-Id ([40 20 35]): 514
laMTG ([73 62 26]): 478
lpFFA ([69 41 26]): 444
lEV-Id ([50 16 34]): 514
ldPreCun ([48 30 51]): 436
lFFA-Id ([66 34 28]): 515
laFFA ([66 58 24]): 515
lpMTG ([78 39 38]): 337
rOFA-Id ([34 23 30]): 515
lIFG ([66 85 40]): 515
raMTG ([14 56 30]): 467
rATL-Id ([27 66 20]): 515
raFFA ([25 57 21]): 468
rpFFA ([26 44 24]): 414
rTPJ ([19 37 47]): 515
lvPreCun ([47 35 48]): 367
rdPreCun-Id ([42 31 61]): 436
lmMTG ([77 49 33]): 418
rmMTG ([15 46 35]): 515
rvPreCun ([41 36 47]): 485
rFFA-Id ([23 31 29]): 508
lOFA-Id ([61 23 29]): 509
lTPJ ([74 32 48]): 424
rpMTG ([13 39 36]): 428
ldPreCun-Id ([48 30 61]): 436
lIFG-Id ([68 82 34]): 509
rdPreCun ([43 30 52]): 367


In [43]:
ds_template = fmri_dataset('/usr/share/data/fsl-mni152-templates/MNI152_T1_2mm_brain.nii.gz')

In [44]:
ds_template = ds_template[:, mask_]

In [45]:
ds_labels = np.zeros((len(rois_), ds_fam.nfeatures), dtype=int)
labels = []
for i, (roi, (center, roi_ids)) in enumerate(rois_.iteritems()):
    ds_labels[i, roi_ids] = 1
    labels.append(roi)

In [46]:
ds_labels_ = Dataset(ds_labels, sa={'labels': labels}, a=ds_template.a)

In [47]:
map2nifti(ds_labels_).to_filename('rois_manual_r{0}_20170102.nii.gz'.format(radius_max))

In [48]:
# save a .1D file containing LPI coordinates to use whereami with AFNI
affine = ds_fam.a.imgaffine
def ijk2xyz(ijk):
    ijk1 = np.hstack((ijk, [1]))
    xyz = np.dot(affine, ijk1)[:3].astype(int)
    return xyz

In [49]:
import pandas as pd

In [50]:
rois_labels = []
rois_xyz = []
for roi, (center, roi_ids) in rois_.iteritems():
    rois_labels.append(roi)
    rois_xyz.append(ijk2xyz(center))

In [51]:
df = pd.DataFrame(rois_xyz, index=rois_labels)

In [52]:
df = df.sort_values([1, 0])

In [53]:
df.to_csv('roi_coord.1D', header=False, sep=' ')

In [54]:
df

,0,1,2
lEV-Id,-10,-94,-4
lLingual,-4,-86,-6
rEV-Id,10,-86,-2
lOFA-Id,-32,-80,-14
rOFA-Id,22,-80,-12
rLingual,6,-76,-4
ldPreCun,-6,-66,30
ldPreCun-Id,-6,-66,50
rdPreCun,4,-66,32
rdPreCun-Id,6,-64,50
